In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

C:\Users\Armughan.Shahid\AppData\Local\Continuum\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [ ]:
class CNN_Model():
    def __init__(self,params):
        input_shape=params['input_shape']
        num_outputs=params['num_outputs']
        fully_connected_hierarchy=params['fc-hierarchy']
        conv_hierarchy=params['conv_hierarchy']
        activation_fn=params.get('activation_fn',tf.nn.relu)
        loss_fn=params.get('loss_fn',tf.losses.softmax_cross_entropy)
        learning_rate=params['learning_rate']
        optimizer_fn=params['optimizer_fn']
#         logdir=params['logdir']
        self.form_model()
    def form_placeholders(self,input_shape,dt=tf.float32):
        X=tf.placeholder(dt,shape=input_shape)
        return X
    def form_convolutional_heirarchy(self,inputs,initializer):
        layer_input=inputs
        for layer in self.conv_hierachy:
            if layer['layer_type']=='conv_layer':
                layer_output=tf.layers.conv2d(
                        inputs=layer_input,
                        filters=layer['num_filters'],
                        kernel_size=layer['kernel_size'],
                        strides=layer['kernel_strides'],
                        padding=layer['padding'],
                        kernel_initializer=initializer,
                        activation=self.activation_fn
                )
                layer_input=layer_output
            elif layer['layer_type']=='pool_layer':
                layer_output=tf.layers.max_pooling2d(
                        inputs=layer_input,
                        pool_size=layer['pool_size'],
                        strides=layer['pool_strides'])
                layer_input=layer_output
        return layer_output
    
    def form_fc_layers(self,inputs,initializer):
        layer_inputs=inputs
        for num_neurons in self.fc_hierarchy:
            layer_outputs=tf.layers.dense(layer_inputs,num_neurons,activation=self.activation_fn,kernel_initializer=initializer)
            layer_inputs=layer_outputs
        return layer_outputs
    def form_loss(self,logits,targets):
        entropies=self.loss_fn(onehot_labels=targets,logits=logits)
        return entropies
    
    def form_model(self):
        initializer_fn=tf.contrib.layers.variance_scaling_initializer()
        self.X=form_placeholders(input_shape)
        self.Y=form_placeholders((1,self.num_outputs),tf.int32)
        conv_output=form_convolutional_heirarchy(self.X,initializer_fn)
        
        flat_conv_output=tf.contrib.layers.flatten(conv_output)
        
        last_fc_output=form_fc_layers(flat_conv_output,initializer_fn)
        
        logits=tf.layers.dense(last_fc_output,self.num_outputs,kernel_initializer=initializer_fn)
        self.logits=logits
#         self.outputs=tf.nn.softmax(self.logits)
#         self.action=tf.multinomial(tf.log(self.outputs),num_samples=1)
        
#         targets=self.action[:][0]
        
        self.entropies=form_loss(logits,targets,loss_fn)
        

In [ ]:
params={
    'input_shape':[None,*input_shape],
    'num_outputs':env.action_space.n,
    'num_neurons':[50],
    'conv_hierarchy':[
        {'layer_type':'conv_layer','kernel_size':4,'kernel_strides':1,'num_filters':5,'padding':'valid'},
#         {'layer_type':'pool_layer','pool_size':3,'pool_strides':1},
        {'layer_type':'conv_layer','kernel_size':4,'kernel_strides':1,'num_filters':10,'padding':'valid'},
        
#         {'layer_type':'conv_layer','kernel_size':3,'kernel_strides':1,'num_filters':10,'padding':'valid'},
#         {'layer_type':'pool_layer','pool_size':3,'pool_strides':1}
       
    ],
    'activation_fn':tf.nn.relu,
    'loss_fn':tf.nn.sparse_softmax_cross_entropy_with_logits,
    'learning_rate':0.01,
    'optimizer_fn':tf.train.AdamOptimizer,
    'logdir':'/tf_logs_rnn/run/'
}